# Проект: English Movie Level Prediction #

### Подготовка словаря английских слов по уровням ###
---

#### Входные данные ####
Электронная таблица. Названия вкладок соответствуют названиям уровней CEFR. На каждой вкладке столбец со словами соответствующего уровня. Слова взяты из файлов "American_Oxford_3000_by_CEFR_level.pdf" и "American_Oxford_5000_by_CEFR_level.pdf" в папке Source/Oxford_CEFR_level

#### Цель ####
Объединить слова в csv-файл, содержащий две колонки: 'Word' и 'Level'.


#### Задачи ####
- если одинаковое написание слова встречается в разных уровнях, присвоить слову наибольший (трудный) уровень.
- сгенерировать тексты с использованием исключительно слов одного уровня (для создания baseline-модели).

## Imports, constants, etc.

In [10]:
# #Install neede libraries
# !pip install pandas
# !pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [5]:
import pandas as pd
import warnings

In [6]:
PATH_LOCAL = 'Source/'                               # local path to data
PATH_REMOTE = '/Source/'                             # remote path to data
CR = '\n'

In [7]:
# Pandas defaults
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.colheader_justify = 'left'

In [8]:
warnings.filterwarnings('ignore')

In [11]:
import PyPDF2
import re

def first_word(word):
    return word.split(maxsplit=1)[0].split(',')[0]

def pdf_parser(name):
    pdf_file = open(f'{PATH_LOCAL}{name}', 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    text = ''
    for page in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page].extract_text()
    lines = text.split('\n')
    data = []
    level_pattern = re.compile(r"^[A|B|C][0-9]{1}")

    current_level = None
    for line in lines:
        if level_pattern.match(line):
            current_level = line.strip()
        else:
            if current_level:
                word = line.split(' ')[0].strip().lower()
                if word != '1':
                    data.append([word, current_level])

    # Create a pandas DataFrame from the data
    df = pd.DataFrame(data, columns=['Word', 'Level'])

    # Apply the custom function to the 'Word' column
    df['Word'] = df['Word'].apply(first_word)
    pdf_file.close()
    return df

In [12]:
oxford_3000 = pdf_parser('Oxford_CEFR_level/The_Oxford_3000_by_CEFR_level.pdf')
oxford_5000 = pdf_parser('Oxford_CEFR_level/The_Oxford_5000_by_CEFR_level.pdf')
am_oxford_3000 = pdf_parser('Oxford_CEFR_level/American_Oxford_3000_by_CEFR_level.pdf')
am_oxford_5000 = pdf_parser('Oxford_CEFR_level/American_Oxford_5000_by_CEFR_level.pdf')

In [13]:
def highest_level(levels):
    level_order = {'A1': 1, 'A2': 2, 'B1': 3, 'B2': 4, 'C1': 5, 'C2': 6}
    return max(levels, key=lambda x: level_order[x])

def comb(df1,df2):
    combined_df = pd.concat([df1, df2])
    # Group the DataFrame by the 'Word' column and apply the custom function to the 'Level' column
    result_df = combined_df.groupby('Word').agg({'Level': highest_level}).reset_index()
    return result_df

In [14]:
oxford_3000.shape

(3263, 2)

In [15]:
oxford_5000.shape

(1798, 2)

In [16]:
oxford = comb(oxford_3000,oxford_5000)
am_oxford = comb(am_oxford_3000,am_oxford_5000)
all = comb(am_oxford,oxford)

In [17]:
all.shape

(4910, 2)

In [18]:
all.shape

(4910, 2)

In [19]:
all.groupby('Level').count()

,Word
Level,
A1,742
A2,751
B1,766
B2,1451
C1,1200


In [20]:
# Save the DataFrame to a CSV file
all.to_csv('words_and_levels.csv', index=False)